In [3]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00


In [27]:
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import torch

In [5]:
dataset = load_dataset('har1/MTS_Dialogue-Clinical_Note')
summarizer = pipeline('summarization', model='facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.67k [00:00<?, ?B/s]

MTS-Dialog-TrainingSet (SDHP).csv:   0%|          | 0.00/1.02M [00:00<?, ?B/s]

MTS-Dialog-Validation Set (SDHP).csv:   0%|          | 0.00/74.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1301 [00:00<?, ? examples/s]

...

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
dataset['train'][0]

{'ID': 0,
 'section_header': 'GENHX',
 'section_text': 'Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.\nDiagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones\nHistory of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable\nPlan of Action: N/A',
 'dialogue': 'Doctor: What brings you back into the clinic today, miss? \nPatient: I came in for a refill of my blood pressure medicine. \nDoctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones.  Have you noticed any changes or do you have any concerns regarding these issues?  \nPatient: No. \nDoctor: Have you had any fever or chills, cough, congestion, nausea, vomiting, chest pain, chest pressure?\nPatient: No.  \nDoc

In [8]:
summary = summarizer(dataset['train']['dialogue'][0])

In [34]:
type(summary)

list

In [35]:
print(dataset['train'][0]['section_text'])
print('')
print(summary[0]['summary_text']) # terrible

Symptoms: no fever, no chills, no cough, no congestion, no nausea, no vomiting, no chest pain, no chest pressure.
Diagnosis: hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis, kidney stones
History of Patient: 76-year-old white female, presents to the clinic today originally for hypertension and a med check, followed by Dr. Kumar, issues stable
Plan of Action: N/A

Patient: I came in for a refill of my blood pressure medicine. Doctor: What brings you back into the clinic today, miss? Patient: I am seventy-six years old and identify as a white female.Doctor: Great. Also, for our records, how old are you and what race do you identify yourself as?


In [14]:
tokenizer = AutoTokenizer.from_pretrained('facebook/bart-large-cnn')
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/bart-large-cnn')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [18]:
tokenized_text = tokenizer(dataset['train'][0]['dialogue'], return_tensors='pt')

In [19]:
tokenized_text

{'input_ids': tensor([[    0, 41152,    35,   653,  3291,    47,   124,    88,     5,  8474,
           452,     6,  2649,   116,  1437, 50118, 18276,  4843,    35,    38,
           376,    11,    13,    10, 40453,     9,   127,  1925,  1164,  6150,
             4,  1437, 50118, 41152,    35,    85,  1326,   101, 12521,  6706,
          1432,    62,    19,    47,    94,    86,  2624,   110, 30960,     6,
         30311,   139, 16880, 37830,     6, 30311,  1517,   368, 13310,     6,
         15671, 37994, 36866,  1809,     6, 28349, 16448,   179, 10100,     8,
         12855, 15109,     4,  1437,  6319,    47,  5324,   143,  1022,    50,
           109,    47,    33,   143,  1379,  2624,   209,   743,   116,  1437,
          1437, 50118, 18276,  4843,    35,   440,     4,  1437, 50118, 41152,
            35,  6319,    47,    56,   143, 11696,    50,  1855,  5622,     6,
         21768,     6, 15744,     6, 27214,     6, 23600,     6,  7050,  2400,
             6,  7050,  1164,   116, 5

In [36]:
output = model.generate(**tokenized_text, num_beams=4, max_length=150, early_stopping=True)
decoded = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded)

Doctor: What brings you back into the clinic today, miss?Patient: I came in for a refill of my blood pressure medicine.Doctor: It looks like Doctor Kumar followed up with you last time regarding your hypertension, osteoarthritis, osteoporosis, hypothyroidism, allergic rhinitis and kidney stones. Have you noticed any changes or do you have any concerns regarding these issues?
